In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
import keras
from tensorflow.keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import *
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

In [6]:
data = pd.read_csv("/content/drive/MyDrive/Data Science/IMDB_movie_review/IMDB Dataset.csv")

In [7]:
def remove_tags(strings):
  result = re.sub('<.*?>','', strings)
  return result
data['review'] = data['review'].apply(lambda x: x.lower())
data['review'] = data['review'].apply(lambda x: remove_tags(x))

In [8]:
data.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [9]:
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(data['review'].values)
X = tokenizer.texts_to_sequences(data['review'].values)
X = pad_sequences(X)

In [10]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(33500, 1853) (33500, 2)
(16500, 1853) (16500, 2)


In [11]:
dropout_rate = 0.3
batch_size = 1000
activation_func = keras.activations.relu

schema = [
          Embedding(5000, 10, input_length=120),
          LSTM(32),
          Dropout(dropout_rate),
          Dense(32, activation=activation_func),
          Dropout(dropout_rate),
          Dense(2 , activation=keras.activations.softmax)
]

model = keras.Sequential(schema)
model.compile(
    optimizer=keras.optimizers.Adam() ,
    loss=keras.losses.categorical_crossentropy ,
    metrics=[ 'accuracy']
)

In [14]:
model.fit(X_train,Y_train,batch_size=1000, epochs=10)

Epoch 1/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2646 - accuracy: 0.8977
Epoch 2/10
34/34 [==============================] - 5s 160ms/step - loss: 0.2570 - accuracy: 0.9012
Epoch 3/10
34/34 [==============================] - 5s 162ms/step - loss: 0.2552 - accuracy: 0.9020
Epoch 4/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2515 - accuracy: 0.9030
Epoch 5/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2493 - accuracy: 0.9054
Epoch 6/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2486 - accuracy: 0.9054
Epoch 7/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2465 - accuracy: 0.9066
Epoch 8/10
34/34 [==============================] - 6s 162ms/step - loss: 0.2438 - accuracy: 0.9072
Epoch 9/10
34/34 [==============================] - 5s 161ms/step - loss: 0.2424 - accuracy: 0.9078
Epoch 10/10
34/34 [==============================] - 5s 160ms/step - loss: 0.2420 - accuracy: 0.9082

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 10)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 56,626
Trainable params: 56,626
Non-trainable params: 0
____________________________________________________

In [17]:
model.evaluate( X_test , Y_test ) 

516/516 [==============================] - 15s 29ms/step - loss: 0.2820 - accuracy: 0.8848


[0.28198808431625366, 0.8848484754562378]

In [26]:
!mkdir -p saved_model
model.save('/content/drive/MyDrive/Data Science/IMDB_movie_review/final_model')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Data Science/IMDB_movie_review/final_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Data Science/IMDB_movie_review/final_model/assets


In [19]:
new_model = keras.models.load_model('saved_model/my_model')
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 10)           50000     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 66        
Total params: 56,626
Trainable params: 56,626
Non-trainable params: 0
____________________________________________________

In [20]:
new_model.evaluate( X_test , Y_test ) 

516/516 [==============================] - 15s 29ms/step - loss: 0.2820 - accuracy: 0.8848


[0.28198808431625366, 0.8848484754562378]

In [23]:
twt = 'That movie was soo bad.'
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = (pad_sequencestwt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = new_model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  11 16 12 73]]
1/1 - 0s
negative
